In [1]:
# Version with gpu.
# frozen VGG
import torch
import torchvision
from torch.nn.utils.rnn import pack_padded_sequence
import nltk
import copy
from PIL import Image
import math
nltk.download('punkt')
import json
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "datasets/captioning/"
train_captions = data_dir+"annotations/captions_train2014.json"
val_captions = data_dir+ "annotations/captions_val2014.json"
train_images_dir = data_dir+"train2014/"
val_images_dir = data_dir+"val2014/"
output_dir = "frozen_heaven2/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir) #thanks jingyu for dis SIK TIP
    
#############################################
total_epochs = 30
embedding_size = 50 #hyper parameters
lr = 0.001 # uh something small.
momentum = 0.9 # the usual
num_layers = 2
num_hidden = 300
batch_size =16
loss_fn = torch.nn.CrossEntropyLoss() # didn't put a softmax layer..
num_workers =0
decay_rate = 0.9
train_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(300), #resized to 300.
        torchvision.transforms.RandomCrop(224), # needs to be this size.
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(), 
        torchvision.transforms.Normalize((0.485, 0.456, 0.406),  # the typical. no 5 crop because we don't want to wait..
                                         (0.229, 0.224, 0.225))])

test_transform = torchvision.transforms.Compose([ 
        torchvision.transforms.Resize(224),# needs to be this size.
        torchvision.transforms.CenterCrop(224), # needs to be this size.
        torchvision.transforms.ToTensor(), 
        torchvision.transforms.Normalize((0.485, 0.456, 0.406),  # the typical.
                                         (0.229, 0.224, 0.225))])

#############################################

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# a = json.load(open(val_captions))
# keys = list(a.keys())
# print(keys)
# print(a[keys[3]][0])
def extract_jsons(file):
    a = json.load(file)
    hold_dict = {}
    return_list = []
    caption_count=0 # so you actually have multiple captions per image. this is to capture them.
    for image_dict in a["images"]:
#         print(image_dict)
#         print(image_dict["file_name"])
#         print(image_dict["id"])
        hold_dict[image_dict["id"]] = [image_dict["file_name"]]#,image_dict["coco_url"]]
    for image_annotations in a["annotations"]:
#         print(image_annotations["image_id"])
#         print(image_annotations["id"])
#         print(image_annotations["caption"])
        hold_dict[image_annotations["image_id"]].append(image_annotations["caption"])
    for image_id in hold_dict.keys():
        image = hold_dict[image_id].pop(0)
        for caption in hold_dict[image_id]:
            return_list.extend([(image,caption)])
    return return_list
val_data = extract_jsons(open(val_captions))
train_data = extract_jsons(open(train_captions))
print(val_data[0]) # tuple. ( image file name, caption)

('COCO_val2014_000000391895.jpg', 'A man with a red helmet on a small moped on a dirt road. ')


In [3]:
# vocab_word = {}
# vocab_id={}
# idcounter =0
# # # this takes a very very very very long time :c
# for item in train_data:
#     latest = nltk.word_tokenize(item[1]) # tokenize to obtain special words and characters...
#     for i in latest: 
#         if i not in vocab_word.keys():
#             vocab_word[i] = idcounter # append to the dictionary.
#             vocab_id[idcounter] = i 
#             idcounter+=1 # increment the id counter
# for item in val_data:
#     latest = nltk.word_tokenize(item[1]) # tokenize to obtain special words and characters...
#     for i in latest: 
#         if i not in vocab_word.keys():
#             vocab_word[i] = idcounter # append to the dictionary.
#             vocab_id[idcounter] = i 
#             idcounter+=1 # increment the id counter            
# for specials in ("<START>","<STOP>","<UNK>"):
#     vocab_word[specials] = idcounter
#     vocab_id[idcounter] = specials
#     idcounter+=1
    
    

# json.dump(vocab_word,open("binderword2id_map.json","w"))# dump so i never have to wait again
# json.dump(vocab_id,open("binderid2wordmap.json","w")) # also pass this to them. or you can make them run it and laugh
vocab_word = json.load(open("eda_caption/word2id_new.json","r"))
vocab_id = json.load(open("eda_caption/id2word_new.json","r"))
# you enjoy right.

In [4]:
print(len(list(vocab_word.keys())))
print(len(list(vocab_id.keys())))
print(list(vocab_id.keys())[0:10])
print(vocab_id["1"])
print(list(vocab_word.keys())[0:50])

21259
21259
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
restaurant
['A', 'restaurant', 'has', 'modern', 'wooden', 'tables', 'and', 'chairs', '.', 'long', 'table', 'with', 'rattan', 'rounded', 'back', 'a', 'plant', 'on', 'top', 'of', 'it', 'surrounded', 'flower', 'arrangement', 'in', 'the', 'middle', 'for', 'is', 'adorned', 'blue', 'accents', 'man', 'preparing', 'desserts', 'kitchen', 'covered', 'frosting', 'chef', 'decorating', 'many', 'small', 'pastries', 'baker', 'prepares', 'various', 'types', 'baked', 'goods', 'close']


In [ ]:
class data_set(torch.utils.data.Dataset):
    def __init__(self,listy,images_dir,transform,vocab_word,vocab_id):
        self.image_dir = images_dir
        self.transform = transform
        self.data = listy # our data is a list. how.. quaint.
        self.word2id = vocab_word
        self.id2word = vocab_id # renamed for better referencing
    def __len__(self):
        return len(self.data)
    def __getitem__(self,index):
        image_name, caption = self.data[index]
        image = self.transform(Image.open(self.image_dir+image_name)) # read the image and transform.
        if image.shape[0]==1:
            image= image = self.transform(Image.open(self.image_dir+image_name).convert("RGB"))
        caption = nltk.word_tokenize(caption)
        caption_list = []
        caption_list.append(self.word2id["<START>"]) # tape on the start
        for i in caption:
            if i in self.word2id.keys():
                caption_list.append(self.word2id[i])
            else:
                caption_list.append(self.word2id["<UNK>"]) # well. i don't know either.
        caption_list.append(self.word2id["<STOP>"]) # tape on the end
        return image, torch.tensor(caption_list)
        

def redo_collation(data):
    #Since pytorch's collate function fucks up, I'll reassign it.
    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[1]), reverse=True) # a lamda to sort based off the length of the tuple.
    # Tuple was from image, caption_list of __getitem__
    images, captions = zip(*data)
    # basically now take apart the tuple
    try:
        images = torch.stack(images, dim = 0) # stack across dimension 0
    except RuntimeError:
        for i in images:
            print (i.shape)
    lengths = torch.tensor([len(cap) for cap in captions]) #obtain lengths
    return_lengths = torch.tensor([len(cap)-1 for cap in captions]) 
    targets = torch.zeros(len(captions), max(lengths)-1) # create empty array.
    input_capt = torch.zeros(len(captions), max(lengths)-1) # create empty array.
    for i in range (len(captions)):
        targets[i,:len(captions[i])-1] = captions[i][1:]#attach accordingly. to main array.
        input_capt[i,:len(captions[i])-1] = captions[i][:-1]
        # this is your target output
        # now you can pad it. after you get the image outputs.
#     print(lengths.shape)
#     print(targets.shape)
#     print(images.shape)
    return images, targets, return_lengths,input_capt
holdy = data_set(val_data,val_images_dir,test_transform,vocab_word,vocab_id)
val_loader = torch.utils.data.DataLoader(holdy, batch_size=batch_size, shuffle=True,num_workers=0, collate_fn=redo_collation)
holdy = data_set(train_data,train_images_dir,train_transform,vocab_word,vocab_id)
train_loader = torch.utils.data.DataLoader(holdy, batch_size=batch_size, shuffle=True,num_workers=0, collate_fn=redo_collation)

# that's for testing. because i forgot about greyscale images..
# for k in range(50):
#     for i in enumerate(val_loader):
#         print("val_pass")
#         print(len(i))
#         break
#     for i in enumerate(train_loader):
#         print(len(i))
#         print("train_pass")
#         break

In [ ]:
class Complete(torch.nn.Module):
    def __init__(self,embed_size,total_words,num_hidden,num_layers):
        super(Complete,self).__init__()
        #self.resnet = torchvision.models.resnet18(pretrained=True)
        self.resnet = torchvision.models.vgg19_bn(pretrained=True)
        self.resnet.classifier[6] = torch.nn.Linear(self.resnet.classifier[6].in_features,num_hidden) #change to different thing
        for param in self.resnet.features.parameters():
            param.requires_grad = False
        for param in self.resnet.classifier[6].parameters():
            param.requires_grad = True
        #self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features,embed_size) #output to same dim as embeddings.
        self.batch_norm = torch.nn.BatchNorm1d(num_hidden) #for smoothness. Also 1d because its 3dim i fucked this up :C
        self.embed = torch.nn.Embedding(total_words,embed_size) #embedding, takes in the number of words...
        #https://www.quora.com/How-do-I-determine-the-number-of-dimensions-for-word-embedding
        #essentially hyper parameter
        self.num_layers = num_layers
        self.hidden = num_hidden
        
        self.lstm = torch.nn.LSTM(embed_size, num_hidden,num_layers,batch_first=True) # ensure batch,seq,word
        self.last = torch.nn.Linear(num_hidden,total_words) #need to output same amount of words as vocab size.
        # return here.
        
    def forward(self,image,caption,length): 
        out1 = self.resnet(image) #resnet the image, getting your outputs.. linear was .. edited
        # maybe i should only toggle the linear layer? leave resnet itself untouched...?
#         print(caption.shape)
        out2 = self.batch_norm(out1) # batchnorm result torch.Size([32, 50]) # this is the result of your resnet
#         print("original caption shape" + str(caption.shape))
        caption = self.embed(caption) # embed the linear output. torch.Size([32, 19, 50]) # these are your captions..
        # you should concat along dimension1. but you need to unsqueeze batchnorm to 32,1,50
#         out2 = out2.unsqueeze(1)
#         out3 = torch.cat((out2,caption),dim = 1) # you concatenate along the dim 1 because dim 0 is batches.
        hiddens = self.initHidden(image.shape[0])
#         out2 = out2.unsqueeze(0)
        holding_list = []
        for i in range(self.num_layers):
            holding_list.append(out2)
        hiddens[0] = torch.stack(holding_list,dim=0) # stack to make images fed into network
        out3 = caption
#         print("Hidden:")
#         print(hiddens[0].shape)
#         print("cellstate:")
#         print(hiddens[1].shape)
#         print("out3.shape:")
#         print(out3.shape)
        # batch,seq, whatever # out3 shape = batch,longestseq, embedding size
        lstm_input = torch.nn.utils.rnn.pack_padded_sequence(out3,length, batch_first = True)
        # essentially...
        # pack = torch.nn.utils.rnn.pack_padded_sequence(batch_in, seq_lengths)
#         try:
        out4,cell_state_nonsense = self.lstm(lstm_input,hiddens)
#         except RuntimeError as e:
#             print("out2.shape : "+ str(out2.shape))
#             print("caption.shape after embed: "+ str(caption.shape))
#             print("length.shape : "+ str(length.shape))
#             print("out3.shape : "+ str(out3.shape))
#             print(str(e))
#             print(length)
#             print(caption)
#             output, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_input, batch_first=True)

        # so here's the funny thing. you apparently put both image and caption in as inputs for training after
        # concatenation.
        # i don't get the math
        # out4 = batch,seq, hidden_dims except you can't see dis shape since it is packed.
#         output, _ = torch.nn.utils.rnn.pad_packed_sequence(out4, batch_first=True)
        out5 = self.last(out4[0]) 
        return out5
#https://discuss.pytorch.org/t/simple-working-example-how-to-use-packing-for-variable-length-sequence-inputs-for-rnn/2120
    
    def caption(self, trigger,image, max_length): #ensure image is indeed of a single batch only. i.e. [1, 3,255]!
        #fuckit you get it
        words =[]
        out1 = self.resnet(image) #process all images
        out2 = self.batch_norm(out1)
        # input to lstm needs to be [1,1,50]
        trigger = self.embed(trigger).unsqueeze(0).unsqueeze(0)
        print(trigger.shape)
        hiddens = self.initHidden(1)
#         out2 = out2.unsqueeze(0)
        holding_list = []
        for i in range(self.num_layers):
            holding_list.append(out2)
        hiddens[0] = torch.stack(holding_list,dim=0) # stack to make  images fed into network
        for i in range(max_length): # maxlength is a hyper parameter here, since you don't know when it's optimal to cut...
            # better extra then nothing to look at at all though...    
#             print(out2.shape)
#             print(hiddens[0].shape)
#             print(hiddens[1].shape)
            out3,states = self.lstm(trigger,hiddens) # amazing how you actually don't put in anything. i don't get math
            out4 = self.last(out3)
            out4 = out4.squeeze(0)
#             print(out4.shape) [1, vocabsize]
#             if greed:
            _ ,sampled = torch.max(out4,dim =1) #look at dimension 1, since now it's just 1, vocab 
#             print(sampled.shape)
            trigger = self.embed(sampled.unsqueeze(0).long())
#             print(trigger.shape)
            # greedy method. try cat?
            # value,indices
#             else:
#                 sampled = int(torch.distributions.categorical.Categorical(torch.exp(out4.squeeze(0))).sample())
                #catty
            words.append(sampled)
            out2 = self.embed(sampled).squeeze(1) # and then you put this in as the next input. yes. the word.
            # i don't get it either.
            
        return words #batch_size, max_seq_length here
    def initHidden(self,batch_size):
        return ([torch.zeros(self.num_layers,batch_size, self.hidden).to(device),torch.zeros(self.num_layers,batch_size, self.hidden).to(device)])

In [ ]:
model =Complete(embedding_size,len(list(vocab_word.keys())),num_hidden,num_layers)
optimizer = torch.optim.SGD(model.parameters(),lr = lr,momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,decay_rate,last_epoch =-1)
#torch.save(model.state_dict(), PATH)
#model = TheModelClass(*args, **kwargs)
model = model.to(device)
#model.load_state_dict(torch.load(PATH))
Train_loss = []
Val_loss = []
Accuracies =[]
train_length = math.ceil(len(train_loader)/batch_size)
val_length = math.ceil(len(val_loader)/batch_size)
print("train length: "+ str(train_length))
print("val length: "+ str(val_length))

train length: 1618
val length: 792


In [ ]:
current_epoch = 15
if current_epoch!=0:
    
    Val_loss = json.load(open(output_dir+"val_loss.json"))
    Accuracies = json.load(open(output_dir+"val_accuracies.json"))
    Train_loss = json.load(open(output_dir+f"train_loss.json"))
    model.load_state_dict(torch.load(output_dir+f"Captioner_{current_epoch-1}.pt"))

for epoch_num in range(current_epoch,total_epochs):
    model.train()
    total_loss =0
    print("Beginning Training")
    
    for idx,(image,target_capt,seqlen,input_capt) in enumerate(train_loader):
        target_capt = torch.nn.utils.rnn.pack_padded_sequence(target_capt, seqlen, batch_first=True)[0]
#         print(image.shape)  # torch.Size([32, 3, 224, 224])
#         print(caption.shape)  # torch.Size([32, ???])
#         print(seqlen.shape)  # torch.Size([32])
        # 32 is technically batchsize
        #??? is the longest tensor's length for captions.
        #feed in caption here.
        if image.shape[0]==1:
            continue
        optimizer.zero_grad()
        output = model(image.float().to(device),input_capt.long().to(device),seqlen.float().to(device))
#         print(output.shape)
#         print(targets.shape)
        
        target_capt = target_capt.long().to(device)
        loss = loss_fn(output,target_capt)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        if idx% 50==0 and idx!=0:
            print("50 Batches done")
#         break  #uncomment to TEST
        
    Train_loss.append(total_loss)#save
    torch.save(model.state_dict(),output_dir+f"Captioner_{epoch_num}.pt")
    json.dump(Train_loss,open(output_dir+f"train_loss.json","w"))
    print("Training Completed. Train loss, model parameters, dumped.")
    model.eval()
    correct=0
    total_encountered =0
    total_loss =0
    print("Beginning Validation")
    with torch.no_grad():
        for idx,(image,target_capt,seqlen,input_capt) in enumerate(val_loader):  
    #         print(image.shape)  # torch.Size([32, 3, 224, 224])
#             print(target_capt.shape)  # torch.Size([32, ???])
    #         print(seqlen.shape)  # torch.Size([32])
            #return_seq = model.caption(image.float(),seqlen[0]) # if you're doing sampling.. but we aren't. ignore this.
            target_capt = torch.nn.utils.rnn.pack_padded_sequence(target_capt, seqlen, batch_first=True)[0]
            output = model(image.float().to(device),input_capt.long().to(device),seqlen.float().to(device))
#             print(output.shape)
#             print(target_capt.shape)
            target_capt = target_capt.long().to(device)
            loss = loss_fn(output,target_capt)
            total_loss+=loss.item()
            _,output = torch.max(output,dim=1)
    #         print("after")
    #         print(output.shape)
    #         print(output)
    #         print(caption)
            for singular in range(len(target_capt)):
                total_encountered+=1
                if int(target_capt[singular]) == int(output[singular]):
                    correct+=1
                    # individual comparison of whether it was CORRECT.
            if idx% 50==0 and idx!=0:
                print("50 Batches done")
#             break #uncomment to TEST
#         break #uncomment to TEST
        
    Val_loss.append(total_loss) #save
    Accuracies.append(correct/total_encountered)
    json.dump(Val_loss,open(output_dir+"val_loss.json","w"))
    json.dump(Accuracies,open(output_dir+"val_accuracies.json","w"))
    print("Validation Completed. Validation losses and accuracies dumped.")
    scheduler.step()
    

#     torch.save(model.state_dict(),output_dir+f"Captioner_{}.pt")

Beginning Training
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Ba

50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batch

50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batch

50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batches done
50 Batch

In [ ]:
####################### JUST MAKIN SURE SAMPLE WORKSSS
for idx,(image,caption,seqlen) in enumerate(val_loader):
    model.eval()
    image = test_transform(Image.open(val_images_dir+val_data[0][0]))
    trigger_input = vocab_word["<START>"]
    trigger_input = torch.tensor(trigger_input).to(device)
    image =image.unsqueeze(0).to(device)
    output = model.caption(trigger_input,image,30)
    caption =[]
    for i in output:
        caption.append(vocab_id[str(int(i))])
    print(caption)
    break
#A man with a red helmet on a small moped on a dirt road. 

In [ ]:
for idx, (image,caption,seqlen) in enumerate(val_loader):
    print(caption.shape)
    print(model.embed(caption.long().to(device)).shape)
    break